### Импорты

In [73]:
import requests
from datetime import datetime
import telebot
from telebot import types
import sqlite3
import random
import io
import dataframe_image as dfi
from PIL import Image

import numpy as np
import pandas as pd
import psutil
from IPython.display import Image

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go

import kaleido
import json
import matplotlib.pyplot as plt
import seaborn as sns
import plotly 
import plotly.graph_objects as go
import collections
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## черновик

In [74]:
# fig.to_image(format="png", engine="kaleido")

In [75]:
# fig = candles(get_quotes('TSLA', '10d', '15m'))

In [76]:
#Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            
#Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo           
        

In [77]:
"""#получение одной новости из финхаба
def get_news(ticker):
    global hub_token
    today = str(datetime.date(datetime.now()))
    r = requests.get(f'https://finnhub.io/api/v1/company-news?symbol={ticker}&from={today}&to={today}&token={hub_token}')
    headline = r.json()[0]['headline']
    url = r.json()[0]['url']
    return (headline, url)""";

In [78]:
"""#получение списка всех новостей из финхаба (заголовки)
def get_news_headlines(ticker):
    global hub_token
    today = str(datetime.date(datetime.now()))
    r = requests.get(f'https://finnhub.io/api/v1/company-news?symbol={ticker}&from={today}&to={today}&token={hub_token}')
    r = r.json()
    #print(r)
    a = []
    for i in r:
        a.append(i['headline'])
    return a""";

## Бот для тестов

In [79]:
bot = telebot.TeleBot("1834334715:AAEtTI6RuWqg7zMW0uCPJWb5_DfRfQ7UdtI")

In [80]:
# result_photo = candles(get_quotes('TSLA', '10d', '1h')).to_image(format="png", engine="kaleido")

In [81]:
@bot.message_handler(content_types=['text'])
def responser(message, text='а там армяне в нарды играют'):
    try:
        ticker, period, interval = message.text.split()
        if interval in ["1m","2m","5m","15m","30m","60m","90m","1h"]:
            result_photo = candles(get_quotes(ticker, period, interval)).to_image(format="png", engine="kaleido")
            bot.send_photo(message.chat.id, result_photo)
        else:
            result_photo = candles(get_quotes(ticker, period, interval), type2 = True).to_image(format="png", engine="kaleido")
            bot.send_photo(message.chat.id, result_photo)
    except:
        bot.send_message(message.chat.id, text = 'Укажите другие данные')

In [82]:
#bot.polling(none_stop=True)

## функции

### базовые функции

In [83]:
def current_price(symbol):
    data = yf.download(tickers=symbol, period='10m', interval='1m')
    
    return data['Close'][-1]

In [84]:
def get_quotes(ticker, period, interval):
    data = yf.download(tickers= ticker, period= period, interval= interval)
    
    if data.size != 0:
        return data
    else:
        return 'неправильные данные'

In [85]:
def candles(data, type2 = False):
    #declare figure
    fig = go.Figure()

    #Candlestick
    fig.add_trace(go.Candlestick(x=data.index,
                    open=data['Open'],
                    high=data['High'],
                    low=data['Low'],
                    close=data['Close'], name = 'market data'))

    # Add titles
    """fig.update_layout(
        #title='Uber live share price evolution',
        yaxis_title='Курс акций в долларах')
"""
    # Делаем кнопки для навигации
    """ fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=15, label="15m", step="minute", stepmode="backward"),
                #dict(count=45, label="45m", step="minute", stepmode="backward"),
                dict(count=1, label="1h", step="hour", stepmode="backward"),
                dict(count=3, label="3h", step="hour", stepmode="backward"),
                dict(count=1, label="1d", step="day", stepmode="backward"),
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1month", step="month", stepmode="backward"),
                dict(step="all")
            ])
        )
    ) """

    #настраиваем размер графика
    fig.update_layout(autosize=True,
                      width=1000,
                      height=700,)

    #убираем разрывы меежду данными. Чтобы график был сплошной. То есть убираем выходные и часы, когда торговля не идет
    if type2:    
        fig.update_xaxes(
                rangeslider_visible=False,
                rangebreaks=[
                    # NOTE: Below values are bound (not single values), ie. hide x to y
                    dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                    #dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                    # dict(values=["2019-12-25", "2020-12-24"])  # hide holidays (Christmas and New Year's, etc)
                ]
            )
    else:
        fig.update_xaxes(
                rangeslider_visible=False,
                rangebreaks=[
                    # NOTE: Below values are bound (not single values), ie. hide x to y
                    dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                    dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                    # dict(values=["2019-12-25", "2020-12-24"])  # hide holidays (Christmas and New Year's, etc)
                ]
            )

    #Show
    #fig.show()
    
    return fig

### Используем апишки для получения новостей

Я написал две версии. Первая выдает смые адекватные новости, но всего 5 штук максимум (ограничение апи). Если этого мало, лучше использовать вторую

In [86]:
google_token = "ef44e796dd82495d9b0ac4ed58354d70"
hub_token = "c33nijqad3i8edlc7t50"
polygon_token = "ZrbA9bOMxg8Nf2qyV14nzV1DLk65mFip"
fmp_token = '9d74ad2bc0eada1a9ef3204d591431a3'

In [87]:
# Получение списка новостей из fmp. Апишник бесплатно выдает только 5 последних новостей, 
# однако их качество намного выше, чем у остальных апи
def get_news(ticker):
    url = f'https://financialmodelingprep.com/api/v3/stock_news?tickers={ticker}&limit=60&apikey={fmp_token}'
    r = requests.get(url).json()
    news = []
    for i in r:
        #не хочется показывать статьи с плохих сайтов и видео с ютуба
        if i['site'] != 'The Motley Fool' and 'www.youtube.com' not in i['url']:
            news.append((i['title'], i['url']))
    return news[:5]

In [88]:
#получение списка новостей из полигона. До ста новостей вроде бы, но пока установил на 30
def get_news2(ticker):
    global polygon_token
    url = (f'https://api.polygon.io/v2/reference/news?limit=30&order=descending&sort=published_utc&ticker={ticker}&published_utc.gte=2021-04-26&apiKey={polygon_token}')
    r = requests.get(url).json()['results']
    news = []
    for i in r:
        if 'www.fool.com' not in i['article_url']:
            news.append((i['title'], i['article_url']))
    return news

### Функция для преобразования датафрейма в изображение (биты)

In [89]:
def table_to_png(table):
    png = io.BytesIO()
    dfi.export(table, png)
    return png.getvalue()

### получение инфы от yfinance

Функции для вывода в качестве изображений табличек с балансом, оддс (ну типа) и рекомендациями 

In [90]:
def balance_sheet(ticker):
    data = yf.Ticker(ticker).balance_sheet
    return table_to_png(data)

In [91]:
def cashflow(ticker):
    data = yf.Ticker(ticker).cashflow  
    return table_to_png(data)

In [92]:
def recommendations(ticker):
    data = yf.Ticker(ticker).recommendations[-20:]
    return table_to_png(data)

описание компании

In [93]:
def summary(ticker):
    return yf.Ticker(ticker).info['longBusinessSummary']

### Аналитика через fmp

In [94]:
#коэффициенты компаний за последние 5 лет. Там их очень много, надо бы отобрать самые нужные
def fin_ratios(ticker):
    url = f'https://financialmodelingprep.com/api/v3/ratios/{ticker}?limit=5&apikey={fmp_token}'
    r = requests.get(url).json()
    df = pd.DataFrame(index = list(r[0].keys())[3:])
    df.index.name = f'{ticker} financial ratios'
    for i in r:
        df[i['date']] = list(i.values())[3:]
    return df

In [95]:
# главные показатели за 5 лет. Опять же, их очень много и надо бы часть убрать
def key_metrics(ticker):
    url = f'https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?limit=5&apikey={fmp_token}'
    r = requests.get(url).json()
    df = pd.DataFrame(index = list(r[0].keys())[3:])
    df.index.name = f'{ticker} key metrics'
    for i in r:
        df[i['date']] = list(i.values())[3:]
    return df